# pET19K_[MaSp2]32 Model Creation

## 1. Load the python packages

In [1]:
from __future__ import print_function, division, absolute_import

# python imports
import re
from os.path import join
from collections import defaultdict
import pickle
import pandas as pd

# third party imports
import pandas
import tabulate
import cobra
pd.set_option('display.max_columns', 100)
pd.set_option('display.width',100)
pd.set_option('display.max_colwidth',300)

# ECOLIme
import ecolime
from ecolime import (transcription, translation, flat_files, generics, formulas, compartments)

# COBRAme
import cobrame
from cobrame.util import building, mu, me_model_interface
#from cobrame.io.json import save_json_me_model, save_reduced_json_me_model

/usr/lib/python3.6/site-packages/cobra-0.5.11-py3.6-linux-x86_64.egg/cobra/solvers/__init__.py:63: UserWarning: No LP solvers found
  warn("No LP solvers found")
/usr/lib/python3.6/site-packages/cobra-0.5.11-py3.6-linux-x86_64.egg/cobra/io/sbml3.py:24: UserWarning: Install lxml for faster SBML I/O
  warn("Install lxml for faster SBML I/O")
/usr/lib/python3.6/site-packages/cobra-0.5.11-py3.6-linux-x86_64.egg/cobra/io/__init__.py:12: UserWarning: cobra.io.sbml requires libsbml
  warn("cobra.io.sbml requires libsbml")


## 2. Load the iJL1678b model

Load the iJL1678b model in pickle format that is located in the current directory.

In [2]:
with open('iJL1678b.pickle', 'rb') as f:
    me = pickle.load(f)

## 3. Define ME-Model Solve Functions

In [3]:
def solve_me_model(me, max_mu, precision=1e-6, min_mu=0, using_soplex=False,
                  compiled_expressions=None):
    if using_soplex:
        from cobrame.solve.algorithms import binary_search
        binary_search(me, min_mu=min_mu, max_mu=max_mu, debug=True, mu_accuracy=precision,
                      compiled_expressions=compiled_expressions)
    else:
        from qminospy.me1 import ME_NLP1
        # The object containing solveME methods--composite that uses a ME model object 
        me_nlp = ME_NLP1(me, growth_key='mu')
        # Use bisection for now (until the NLP formulation is worked out)
        muopt, hs, xopt, cache = me_nlp.bisectmu(precision=precision, mumax=max_mu)
        me.solution.f = me.solution.x_dict['biomass_dilution']
        
def show_escher_map(me, solution=None):
    import escher
    view = escher.Builder("iJO1366.Central metabolism")
    view.reaction_data = me.get_metabolic_flux(solution=solution)
    return view

## 4. Create the MaSp2 Transcription Reaction

Complete the steps necessary to create the MaSp2 Transcription Reaction

In [4]:
locus_id = 'MaSp2'
left_pos =  None
right_pos = None
seq = 'ATGACCGGTCCGGGTCAGCAGGGTCCGGGTGGTTATGGTCCTGGCCAGCAGGGACCGAGCGGTCCGGGTAGTGCAGCAGCAGCTGCAGCAGCCGCAGGCCCTGGTCAGCAAGGCCCTGGTGGATATGGACCAGGCCAACAGGGTCCTGGCGGATACGGTCCTGGTCAACAAGGTCCGTCAGGTCCGGGTTCAGCCGCAGCGGCTGCTGCCGCAGCAGGTCCAGGTGGCTACGGACCGGGTCAACAGGGACCCGGTGGGTACGGACCAGGACAGCAAGGGCCAGGCGGTTATGGCCCTGGACAACAAGGGCCTAGTGGTCCTGGTTCTGCAGCGGCAGCCGCTGCGGCAGCTGGTCCGGGACAGCAAGGACCCGGTGGATACGGTCCCGGTCAGCAGGGACCTGGCGGTTACGGACCCGGACAACAGGGTCCATCTGGTCCTGGTAGCGCAGCCGCAGCAGCAGCGGCTGCAGGTCCAGGACAACAAGGTCCTGGTGGGTATGGTCCAGGGCAGCAAGGTCCGAGTGGTCCAGGCTCTGCGGCAGCGGCAGCAGCAGCAGCGGGACCTGGTCAACAGGGTCCAGGGGGATATGGCCCAGGTCAGCAAGGACCGGGTGGCTATGGGCCAGGTCAACAAGGCCCTAGCGGTCCGGGATCTGCCGCAGCTGCAGCGGCAGCGGCAGGTCCTGGCGGTTATGGACCAGGTCAGCAGGGTCCCGGTGGCTACGGTCCCGGACAACAAGGCCCAGGGGGTTACGGACCTGGCCAGCAAGGTCCTTCTGGACCGGGAAGCGCTGCAGCCGCAGCAGCTGCAGCCGGTCCAGGCCAGCAAGGGCCTGGGGGTTACGGTCCGGGTCAGCAAGGCCCAGGCGGATACGGTCCAGGACAACAGGGACCAAGTGGTCCGGGATCAGCAGCCGCTGCCGCAGCGGCAGCCGGTCCCGGTCAACAAGGACCTGGTGGCTACGGTCCTGGGCAACAGGGTCCTAGCGGTCCAGGGTCAGCAGCAGCAGCCGCAGCTGCAGCATCCGGTCCGGGTCAGCAGGGTCCGGGTGGTTATGGTCCTGGCCAGCAGGGACCGAGCGGTCCGGGTAGTGCAGCAGCAGCTGCAGCAGCCGCAGGCCCTGGTCAGCAAGGCCCTGGTGGATATGGACCAGGCCAACAGGGTCCTGGCGGATACGGTCCTGGTCAACAAGGTCCGTCAGGTCCGGGTTCAGCCGCAGCGGCTGCTGCCGCAGCAGGTCCAGGTGGCTACGGACCGGGTCAACAGGGACCCGGTGGGTACGGACCAGGACAGCAAGGGCCAGGCGGTTATGGCCCTGGACAACAAGGGCCTAGTGGTCCTGGTTCTGCAGCGGCAGCCGCTGCGGCAGCTGGTCCGGGACAGCAAGGACCCGGTGGATACGGTCCCGGTCAGCAGGGACCTGGCGGTTACGGACCCGGACAACAGGGTCCATCTGGTCCTGGTAGCGCAGCCGCAGCAGCAGCGGCTGCAGGTCCAGGACAACAAGGTCCTGGTGGGTATGGTCCAGGGCAGCAAGGTCCGAGTGGTCCAGGCTCTGCGGCAGCGGCAGCAGCAGCAGCGGGACCTGGTCAACAGGGTCCAGGGGGATATGGCCCAGGTCAGCAAGGACCGGGTGGCTATGGGCCAGGTCAACAAGGCCCTAGCGGTCCGGGATCTGCCGCAGCTGCAGCGGCAGCGGCAGGTCCTGGCGGTTATGGACCAGGTCAGCAGGGTCCCGGTGGCTACGGTCCCGGACAACAAGGCCCAGGGGGTTACGGACCTGGCCAGCAAGGTCCTTCTGGACCGGGAAGCGCTGCAGCCGCAGCAGCTGCAGCCGGTCCAGGCCAGCAAGGGCCTGGGGGTTACGGTCCGGGTCAGCAAGGCCCAGGCGGATACGGTCCAGGACAACAGGGACCAAGTGGTCCGGGATCAGCAGCCGCTGCCGCAGCGGCAGCCGGTCCCGGTCAACAAGGACCTGGTGGCTACGGTCCTGGGCAACAGGGTCCTAGCGGTCCAGGGTCAGCAGCAGCAGCCGCAGCTGCAGCATCCGGTCCGGGTCAGCAGGGTCCGGGTGGTTATGGTCCTGGCCAGCAGGGACCGAGCGGTCCGGGTAGTGCAGCAGCAGCTGCAGCAGCCGCAGGCCCTGGTCAGCAAGGCCCTGGTGGATATGGACCAGGCCAACAGGGTCCTGGCGGATACGGTCCTGGTCAACAAGGTCCGTCAGGTCCGGGTTCAGCCGCAGCGGCTGCTGCCGCAGCAGGTCCAGGTGGCTACGGACCGGGTCAACAGGGACCCGGTGGGTACGGACCAGGACAGCAAGGGCCAGGCGGTTATGGCCCTGGACAACAAGGGCCTAGTGGTCCTGGTTCTGCAGCGGCAGCCGCTGCGGCAGCTGGTCCGGGACAGCAAGGACCCGGTGGATACGGTCCCGGTCAGCAGGGACCTGGCGGTTACGGACCCGGACAACAGGGTCCATCTGGTCCTGGTAGCGCAGCCGCAGCAGCAGCGGCTGCAGGTCCAGGACAACAAGGTCCTGGTGGGTATGGTCCAGGGCAGCAAGGTCCGAGTGGTCCAGGCTCTGCGGCAGCGGCAGCAGCAGCAGCGGGACCTGGTCAACAGGGTCCAGGGGGATATGGCCCAGGTCAGCAAGGACCGGGTGGCTATGGGCCAGGTCAACAAGGCCCTAGCGGTCCGGGATCTGCCGCAGCTGCAGCGGCAGCGGCAGGTCCTGGCGGTTATGGACCAGGTCAGCAGGGTCCCGGTGGCTACGGTCCCGGACAACAAGGCCCAGGGGGTTACGGACCTGGCCAGCAAGGTCCTTCTGGACCGGGAAGCGCTGCAGCCGCAGCAGCTGCAGCCGGTCCAGGCCAGCAAGGGCCTGGGGGTTACGGTCCGGGTCAGCAAGGCCCAGGCGGATACGGTCCAGGACAACAGGGACCAAGTGGTCCGGGATCAGCAGCCGCTGCCGCAGCGGCAGCCGGTCCCGGTCAACAAGGACCTGGTGGCTACGGTCCTGGGCAACAGGGTCCTAGCGGTCCAGGGTCAGCAGCAGCAGCCGCAGCTGCAGCATCCGGTCCGGGTCAGCAGGGTCCGGGTGGTTATGGTCCTGGCCAGCAGGGACCGAGCGGTCCGGGTAGTGCAGCAGCAGCTGCAGCAGCCGCAGGCCCTGGTCAGCAAGGCCCTGGTGGATATGGACCAGGCCAACAGGGTCCTGGCGGATACGGTCCTGGTCAACAAGGTCCGTCAGGTCCGGGTTCAGCCGCAGCGGCTGCTGCCGCAGCAGGTCCAGGTGGCTACGGACCGGGTCAACAGGGACCCGGTGGGTACGGACCAGGACAGCAAGGGCCAGGCGGTTATGGCCCTGGACAACAAGGGCCTAGTGGTCCTGGTTCTGCAGCGGCAGCCGCTGCGGCAGCTGGTCCGGGACAGCAAGGACCCGGTGGATACGGTCCCGGTCAGCAGGGACCTGGCGGTTACGGACCCGGACAACAGGGTCCATCTGGTCCTGGTAGCGCAGCCGCAGCAGCAGCGGCTGCAGGTCCAGGACAACAAGGTCCTGGTGGGTATGGTCCAGGGCAGCAAGGTCCGAGTGGTCCAGGCTCTGCGGCAGCGGCAGCAGCAGCAGCGGGACCTGGTCAACAGGGTCCAGGGGGATATGGCCCAGGTCAGCAAGGACCGGGTGGCTATGGGCCAGGTCAACAAGGCCCTAGCGGTCCGGGATCTGCCGCAGCTGCAGCGGCAGCGGCAGGTCCTGGCGGTTATGGACCAGGTCAGCAGGGTCCCGGTGGCTACGGTCCCGGACAACAAGGCCCAGGGGGTTACGGACCTGGCCAGCAAGGTCCTTCTGGACCGGGAAGCGCTGCAGCCGCAGCAGCTGCAGCCGGTCCAGGCCAGCAAGGGCCTGGGGGTTACGGTCCGGGTCAGCAAGGCCCAGGCGGATACGGTCCAGGACAACAGGGACCAAGTGGTCCGGGATCAGCAGCCGCTGCCGCAGCGGCAGCCGGTCCCGGTCAACAAGGACCTGGTGGCTACGGTCCTGGGCAACAGGGTCCTAGCGGTCCAGGGTCAGCAGCAGCAGCCGCAGCTGCAGCATCCGGAGGG'
strand = None
rna_type = 'mRNA'

building.create_transcribed_gene(me, locus_id, rna_type, seq)
building.add_transcription_reaction(me, 'TU_MaSp2', {'MaSp2'}, seq)
print(me.reactions.transcription_TU_MaSp2.reaction)
me.reactions.transcription_TU_MaSp2.update()

549 atp_c + 1570 ctp_c + 1218 gtp_c + 767 utp_c --> RNA_MaSp2 + 1311.0214523240002 mRNA_biomass + 4104 ppi_c


/source/cobrame/cobrame/core/reaction.py:852 UserWarning: RNA Polymerase () not found


###  Add the RNAP to the MaSp2 Transcription Reaction

In [5]:
me.reactions.transcription_TU_MaSp2.transcription_data.RNA_polymerase = 'RNAP70-CPLX'
me.reactions.transcription_TU_MaSp2.update()

List the transcription data ("transcription_data") associated with the MaSp2 transcription reaction ("transcription_TU_MaSp2").

In [6]:
rxn = me.reactions.transcription_TU_MaSp2
data = rxn.transcription_data
pd.DataFrame({i: str(v) for i, v in data.__dict__.items()}, index=['Attribute Values']).T

,Attribute Values
id,TU_MaSp2
_model,iJL1678b-ME
_parent_reactions,{'transcription_TU_MaSp2'}
nucleotide_sequence,ATGACCGGTCCGGGTCAGCAGGGTCCGGGTGGTTATGGTCCTGGCCAGCAGGGACCGAGCGGTCCGGGTAGTGCAGCAGCAGCTGCAGCAGCCGCAGGCCCTGGTCAGCAAGGCCCTGGTGGATATGGACCAGGCCAACAGGGTCCTGGCGGATACGGTCCTGGTCAACAAGGTCCGTCAGGTCCGGGTTCAGCCGCAGCGGCTGCTGCCGCAGCAGGTCCAGGTGGCTACGGACCGGGTCAACAGGGACCCGGTGGGTACGGACCAGGACAGCAAGGGCCAGGCGGTTATGGCCC...
RNA_products,{'RNA_MaSp2'}
RNA_polymerase,RNAP70-CPLX
subreactions,"defaultdict(<class 'int'>, {})"


List the attributes of the MaSp2 transcription reaction.

In [7]:
rxn = me.reactions.transcription_TU_MaSp2
data = rxn
pd.DataFrame({i: str(v) for i, v in data.__dict__.items()}, index=['Attribute Values']).T

,Attribute Values
id,transcription_TU_MaSp2
name,
notes,{}
annotation,{}
_gene_reaction_rule,
subsystem,
_genes,set()
_metabolites,"{<RNAP RNAP70-CPLX at 0x7f2c7f7ee390>: -0.0196124982794219*mu - 0.00767829307639367, <TranscribedGene RNA_MaSp2 at 0x7f2c7f919518>: 1, <Metabolite utp_c at 0x7f2c7f7eee10>: -767, <Metabolite atp_c at 0x7f2c7f7ee9b0>: -549, <Metabolite ctp_c at 0x7f2c7f7eeef0>: -1570, <Metabolite gtp_c at 0x7f2c7..."
_model,iJL1678b-ME
_objective_coefficient,0.0


List the complete reaction associated with the MaSp2 transcription reaction.

In [8]:
me.reactions.get_by_id('transcription_TU_MaSp2').reaction

'0.0196124982794219*mu + 0.00767829307639367 RNAP70-CPLX + 549 atp_c + 1570 ctp_c + 1218 gtp_c + 767 utp_c --> RNA_MaSp2 + 1311.0214523240002 mRNA_biomass + 4104 ppi_c'

List the RNAP associated with the MaSp2 transcription reaction

In [9]:
me.reactions.transcription_TU_MaSp2.transcription_data.RNA_polymerase

'RNAP70-CPLX'

## 5. Create the MaSp2 Translation Reaction

Complete all the steps necessary to create the MaSp2 Translation Reaction

### Storing information about a translation reaction

In [10]:
data = cobrame.TranslationData('MaSp2', me, 'RNA_MaSp2', 'protein_MaSp2')
data.nucleotide_sequence = seq

### Create translation reaction

By associating the TranslationReaction with its corresponding TranslationData object and running the update function, COBRAme will create a translation reaction for the nucleotide sequence given based on the organisms codon table and prespecified translation machinery.

In [11]:
rxn = cobrame.TranslationReaction('translation_MaSp2')
rxn.translation_data = data
me.add_reaction(rxn)

### Updating the translation reaction

Note that this process does not include the ribosome in the reaction.

In [12]:
rxn.update()
print(rxn.reaction)

0.000498399634202103*mu + 0.000195123456790123 + 0.00598079561042524*(mu + 0.3915)/mu RNA_MaSp2 + 2.55589556001079e-8*mu + 1.00063311174422e-8 RNA_degradosome + 288 ala__L_c + 6.13480109739369*(mu + 0.3915)/mu atp_c + 192 gln__L_c + 506 gly_c + 7.84095134723271*(mu + 0.3915)/mu mRNA_biomass + met__L_c + 236 pro__L_c + 0.0196124982794219*mu + 0.00767829307639367 ribosome + 76 ser__L_c + thr__L_c + 68 tyr__L_c --> 6.13480109739369*(mu + 0.3915)/mu adp_c + 3.28345679012346*(mu + 0.3915)/mu amp_c + 9.38984910836763*(mu + 0.3915)/mu cmp_c + 7.28460905349794*(mu + 0.3915)/mu gmp_c + 1367.0 - 6.13480109739369*(mu + 0.3915)/mu h2o_c + 6.13480109739369*(mu + 0.3915)/mu h_c + 6.13480109739369*(mu + 0.3915)/mu pi_c + protein_MaSp2 + 114.82236754 protein_biomass + 4.58727023319616*(mu + 0.3915)/mu ump_c


### Incorporate the ribosome

Add the ribosome to the reaction.

In [13]:
ribosome = cobrame.Ribosome('ribosome')
me.add_metabolites([ribosome])
me.reactions.translation_MaSp2.update()
print(me.reactions.translation_MaSp2.reaction)

0.000498399634202103*mu + 0.000195123456790123 + 0.00598079561042524*(mu + 0.3915)/mu RNA_MaSp2 + 2.55589556001079e-8*mu + 1.00063311174422e-8 RNA_degradosome + 288 ala__L_c + 6.13480109739369*(mu + 0.3915)/mu atp_c + 192 gln__L_c + 506 gly_c + 7.84095134723271*(mu + 0.3915)/mu mRNA_biomass + met__L_c + 236 pro__L_c + 0.0196124982794219*mu + 0.00767829307639367 ribosome + 76 ser__L_c + thr__L_c + 68 tyr__L_c --> 6.13480109739369*(mu + 0.3915)/mu adp_c + 3.28345679012346*(mu + 0.3915)/mu amp_c + 9.38984910836763*(mu + 0.3915)/mu cmp_c + 7.28460905349794*(mu + 0.3915)/mu gmp_c + 1367.0 - 6.13480109739369*(mu + 0.3915)/mu h2o_c + 6.13480109739369*(mu + 0.3915)/mu h_c + 6.13480109739369*(mu + 0.3915)/mu pi_c + protein_MaSp2 + 114.82236754 protein_biomass + 4.58727023319616*(mu + 0.3915)/mu ump_c


### Add Subreactions to the MaSp2 model

Create the subreactions that represent elongation, initiation and termination

In [14]:
elongation = ['FusA_mono_elongation', 'Tuf_gtp_regeneration']
initiation = ['Translation_gtp_initiation_factor_InfB', 'Translation_initiation_factor_InfA', 
              'Translation_initiation_factor_InfC', 'fmet_addition_at_START', 'peptide_deformylase_processing', 
              'peptide_chain_release', 'ribosome_recycler']
termination = {'UGA':'PrfB_mono'}
me.process_data.get_by_id('MaSp2').add_elongation_subreactions(elongation_subreactions=set(elongation))
me.process_data.get_by_id('MaSp2').add_initiation_subreactions(start_codons=set(['AUG']), start_subreactions=set(initiation))
me.process_data.get_by_id('MaSp2').add_termination_subreactions(translation_terminator_dict=termination)
#me.reactions.translation_MaSp2.update(verbose=False)

/source/cobrame/cobrame/core/processdata.py:896 UserWarning: No termination enzyme for RNA_MaSp2


List the subreactions associated with MaSp2 translation reaction stored in the ProcessData.

In [15]:
me.process_data.get_by_id('MaSp2').subreactions

defaultdict(int,
            {'Tuf_gtp_regeneration': 1367.0,
             'FusA_mono_elongation': 1367.0,
             'thr_addition_at_ACC': 1,
             'gly_addition_at_GGU': 252,
             'pro_addition_at_CCG': 56,
             'gln_addition_at_CAG': 88,
             'tyr_addition_at_UAU': 28,
             'pro_addition_at_CCU': 80,
             'gly_addition_at_GGC': 84,
             'gly_addition_at_GGA': 121,
             'ser_addition_at_AGC': 20,
             'ser_addition_at_AGU': 16,
             'ala_addition_at_GCA': 156,
             'ala_addition_at_GCU': 44,
             'ala_addition_at_GCC': 48,
             'gln_addition_at_CAA': 104,
             'pro_addition_at_CCA': 72,
             'tyr_addition_at_UAC': 40,
             'ser_addition_at_UCA': 16,
             'ala_addition_at_GCG': 40,
             'pro_addition_at_CCC': 28,
             'gly_addition_at_GGG': 48,
             'ser_addition_at_UCU': 20,
             'ser_addition_at_UCC': 4,
           

List the subreactions generated from the nucleatide sequence that are stored in the ProcessData.

In [16]:
me.process_data.get_by_id('MaSp2').subreactions_from_sequence

{'thr_addition_at_ACC': 1,
 'gly_addition_at_GGU': 252,
 'pro_addition_at_CCG': 56,
 'gln_addition_at_CAG': 88,
 'tyr_addition_at_UAU': 28,
 'pro_addition_at_CCU': 80,
 'gly_addition_at_GGC': 84,
 'gly_addition_at_GGA': 121,
 'ser_addition_at_AGC': 20,
 'ser_addition_at_AGU': 16,
 'ala_addition_at_GCA': 156,
 'ala_addition_at_GCU': 44,
 'ala_addition_at_GCC': 48,
 'gln_addition_at_CAA': 104,
 'pro_addition_at_CCA': 72,
 'tyr_addition_at_UAC': 40,
 'ser_addition_at_UCA': 16,
 'ala_addition_at_GCG': 40,
 'pro_addition_at_CCC': 28,
 'gly_addition_at_GGG': 48,
 'ser_addition_at_UCU': 20,
 'ser_addition_at_UCC': 4}

### Updating the MaSp2 translation reaction

In [17]:
translation_subreactions = me.process_data.MaSp2.subreactions_from_sequence
print("Before adding tRNA subreaction")
print("------------------------------")
print(me.reactions.translation_MaSp2.reaction)
print("")
# Link tranlation_data to subreactions and update
for subreaction, value in translation_subreactions.items():
    me.process_data.MaSp2.subreactions[subreaction] = value
me.reactions.translation_MaSp2.update(verbose=False)
print("After adding tRNA subreaction")
print("-----------------------------")
print(me.reactions.translation_MaSp2.reaction)

Before adding tRNA subreaction
------------------------------
0.000498399634202103*mu + 0.000195123456790123 + 0.00598079561042524*(mu + 0.3915)/mu RNA_MaSp2 + 2.55589556001079e-8*mu + 1.00063311174422e-8 RNA_degradosome + 288 ala__L_c + 6.13480109739369*(mu + 0.3915)/mu atp_c + 192 gln__L_c + 506 gly_c + 7.84095134723271*(mu + 0.3915)/mu mRNA_biomass + met__L_c + 236 pro__L_c + 0.0196124982794219*mu + 0.00767829307639367 ribosome + 76 ser__L_c + thr__L_c + 68 tyr__L_c --> 6.13480109739369*(mu + 0.3915)/mu adp_c + 3.28345679012346*(mu + 0.3915)/mu amp_c + 9.38984910836763*(mu + 0.3915)/mu cmp_c + 7.28460905349794*(mu + 0.3915)/mu gmp_c + 1367.0 - 6.13480109739369*(mu + 0.3915)/mu h2o_c + 6.13480109739369*(mu + 0.3915)/mu h_c + 6.13480109739369*(mu + 0.3915)/mu pi_c + protein_MaSp2 + 114.82236754 protein_biomass + 4.58727023319616*(mu + 0.3915)/mu ump_c

After adding tRNA subreaction
-----------------------------
10fthf_c + 2.72438972852433e-7*mu Def_mono_mod_1:fe2 + 1.80324164444648e-7

List the reactants associated with the MaSp2 translation reaction. In this list all the subreactions should be listed.

In [18]:
me.reactions.translation_MaSp2.reactants

[<Metabolite met__L_c at 0x7f2c7eaef978>,
 <Metabolite h2o_c at 0x7f2c7f7eea58>,
 <Metabolite thr__L_c at 0x7f2c7eaefdd8>,
 <Metabolite gly_c at 0x7f2c7eaefcc0>,
 <Metabolite pro__L_c at 0x7f2c7eafd2b0>,
 <Metabolite gln__L_c at 0x7f2c7eafd240>,
 <Metabolite tyr__L_c at 0x7f2c7eafd438>,
 <Metabolite ser__L_c at 0x7f2c7eaefe80>,
 <Metabolite ala__L_c at 0x7f2c7eaeff28>,
 <Ribosome ribosome at 0x7f2c7eafd518>,
 <TranscribedGene RNA_MaSp2 at 0x7f2c7f919518>,
 <Metabolite atp_c at 0x7f2c7f7ee9b0>,
 <Complex RNA_degradosome at 0x7f2c7f8163c8>,
 <Complex Tsf_mono at 0x7f2c7eb17c18>,
 <Complex FusA_mono at 0x7f2c7eb17be0>,
 <Metabolite gtp_c at 0x7f2c7f7eefd0>,
 <GenericComponent generic_Tuf at 0x7f2c7eafda20>,
 <GenerictRNA generic_tRNA_ACC_thr__L_c at 0x7f2c7eb0e6a0>,
 <GenerictRNA generic_tRNA_GGU_gly_c at 0x7f2c7eafdf28>,
 <GenerictRNA generic_tRNA_CCG_pro__L_c at 0x7f2c7eb0ed30>,
 <GenerictRNA generic_tRNA_CAG_gln__L_c at 0x7f2c7eb0e5c0>,
 <GenerictRNA generic_tRNA_UAU_tyr__L_c at 0x7f2c

This completes the creation of the translation reaction

## 6. Create Demand Reaction for MaSp2

In [19]:
new_reaction = cobrame.MEReaction('DM_protein_MaSp2')
me.add_reaction(new_reaction)

In [20]:
me.reactions.get_by_id('DM_protein_MaSp2').reaction = 'protein_MaSp2 ->'
me.reactions.get_by_id('DM_protein_MaSp2').reaction

'protein_MaSp2 --> '

In [21]:
rxn = me.reactions.DM_protein_MaSp2
data = rxn
pd.DataFrame({i: str(v) for i, v in data.__dict__.items()}, index=['Attribute Values']).T

,Attribute Values
id,DM_protein_MaSp2
name,
notes,{}
annotation,{}
_gene_reaction_rule,
subsystem,
_genes,set()
_metabolites,{<TranslatedGene protein_MaSp2 at 0x7f2c7f8e0470>: -1}
_model,iJL1678b-ME
_objective_coefficient,0.0


### Find the amino acid count

In [22]:
me.process_data.MaSp2.amino_acid_count

defaultdict(int,
            {'met__L_c': 1,
             'thr__L_c': 1,
             'gly_c': 506,
             'pro__L_c': 236,
             'gln__L_c': 192,
             'tyr__L_c': 68,
             'ser__L_c': 76,
             'ala__L_c': 288})

In [23]:
import pickle
with open('iJL1678b_pET19K_MaSp2_32.pickle', 'wb') as f:
    pickle.dump(me, f)

In [24]:
with open('iJL1678b_pET19K_MaSp2_32.pickle', 'rb') as f:
    test_me = pickle.load(f)

In [25]:
test_me

<MEModel iJL1678b-ME at 0x7f2c7f0024e0>

In [26]:
test_me.global_info

{'kt': 4.5,
 'r0': 0.087,
 'k_deg': 12.0,
 'm_rr': 1453.0,
 'm_aa': 0.109,
 'f_rRNA': 0.86,
 'm_nt': 0.324,
 'f_mRNA': 0.02,
 'm_tRNA': 25.0,
 'f_tRNA': 0.12,
 'temperature': 37,
 'propensity_scaling': 0.45,
 'GC_fraction': 0.507896997096}